In [3]:
# !pip install underthesea

In [13]:
# !pip install coqpit

In [14]:
# %%cd XTTSv2-Finetuning-for-New-Languages

In [1]:
import torch
import torchaudio
from tqdm import tqdm
from underthesea import sent_tokenize
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

/root/anaconda3/envs/whisper_env/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
# !pip install tts

In [3]:
# !pip install mutagen

In [4]:
!ls checkpoints/GPT_XTTS_FT-April-02-2025_03+47AM-8e59ec3

best_model.pth
best_model_30000.pth
config.json
events.out.tfevents.1743558474.Ubuntu-2404-noble-amd64-base.735363.0
train_gpt_xtts.py
trainer_0_log.txt


In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
xtts_checkpoint = "./checkpoints/GPT_XTTS_FT-April-02-2025_03+47AM-8e59ec3/best_model_30000.pth"
xtts_config = "./checkpoints/GPT_XTTS_FT-April-02-2025_03+47AM-8e59ec3/config.json"
xtts_vocab = "./checkpoints/XTTS_v2.0_original_model_files/vocab.json"

In [3]:
xtts_vocab

'./checkpoints/XTTS_v2.0_original_model_files/vocab.json'

In [4]:
config = XttsConfig()
config.load_json(xtts_config)
XTTS_MODEL = Xtts.init_from_config(config)

Vocab_file None


In [5]:
XTTS_MODEL.load_checkpoint(config, checkpoint_path=xtts_checkpoint, vocab_path=xtts_vocab, use_deepspeed=False)

Vocab_file ./checkpoints/XTTS_v2.0_original_model_files/vocab.json


GPT2InferenceModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [35]:
# !cat checkpoints/XTTS_v2.0_original_model_files/vocab.json

In [6]:
# !pip install tokenizers==0.20.1

In [7]:
# pip install transformers -U

In [10]:
!ls datasets-1

metadata_test.csv  metadata_train.csv  wavs


In [11]:
import pandas as pd

In [15]:
df = pd.read_csv('./datasets-1/metadata_train.csv',delimiter="|")

In [17]:
wave_file = './datasets-1/wavs/vocals_chunk_0181_1306890ms_wip8_24k.wav'

In [32]:
tts_text = f"""اسرائیلی وزیر اعظم نیاتن یاھو ایک منحوس انسان ہے"""
speaker_audio_file = wave_file
lang = "ur"

gpt_cond_latent, speaker_embedding = XTTS_MODEL.get_conditioning_latents(
    audio_path= [speaker_audio_file],
    gpt_cond_len=XTTS_MODEL.config.gpt_cond_len,
    max_ref_length=XTTS_MODEL.config.max_ref_len,
    sound_norm_refs=XTTS_MODEL.config.sound_norm_refs,
)

tts_texts = [tts_text]
wav_chunks = []
for text in tqdm(tts_texts):
    wav_chunk = XTTS_MODEL.inference(
        text=text,
        language=lang,
        gpt_cond_latent=gpt_cond_latent,
        speaker_embedding=speaker_embedding,
        temperature=0.01,
        length_penalty=0.1,
        repetition_penalty=10.0,
        top_k=10,
        top_p=0.3,
    )
    wav_chunks.append(torch.tensor(wav_chunk["wav"]))

out_wav = torch.cat(wav_chunks, dim=0).unsqueeze(0).cpu()

from IPython.display import Audio
Audio(out_wav, rate=24000)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.51s/it]


In [34]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id="xtts-urdu-voice", repo_type="model")

RepoUrl('https://huggingface.co/m-aliabbas1/xtts-urdu-voice', endpoint='https://huggingface.co', repo_type='model', repo_id='m-aliabbas1/xtts-urdu-voice')

Readme.md    download_checkpoint.py  train_dvae_xtts.py  tts_inference.ipynb
TTS	     extend_vocab_config.py  train_dvae_xtts.sh
checkpoints  recipes		     train_gpt_xtts.py
datasets-1   requirements.txt	     train_gpt_xtts.sh


In [45]:
!mkdir xtts-urdu-voice/

mkdir: cannot create directory ‘xtts-urdu-voice/’: File exists


In [39]:
!cp ./checkpoints/GPT_XTTS_FT*/best_model_30000.pth xtts-urdu-voice/
!cp ./checkpoints/GPT_XTTS_FT*/config.json xtts-urdu-voice/
!cp ./checkpoints/XTTS_v2.0_original_model_files/vocab.json xtts-urdu-voice/

In [40]:
!cp ./datasets-1/wavs/vocals_chunk_0181_1306890ms_wip8_24k.wav xtts-urdu-voice/

In [44]:
!mv xtts-urdu-voice/best_model_30000.pth xtts-urdu-voice/model.pth 
!mv xtts-urdu-voice/vocals_chunk_0181_1306890ms_wip8_24k.wav xtts-urdu-voice/sample_wav.wav

In [42]:
import os
os.rename(
    "./checkpoints/GPT_XTTS_FT-April-02-2025_03+47AM-8e59ec3/best_model_30000.pth",
    "./checkpoints/GPT_XTTS_FT-April-02-2025_03+47AM-8e59ec3/model.pth"
)


In [46]:
!ls xtts-urdu-voice/

config.json  model.pth	sample_wav.wav	vocab.json


In [47]:
from huggingface_hub import HfApi, HfFolder, upload_folder

In [50]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id="m-aliabbas1/xtts-urdu-voice-v0", repo_type="model", exist_ok=True)


RepoUrl('https://huggingface.co/m-aliabbas1/xtts-urdu-voice-v0', endpoint='https://huggingface.co', repo_type='model', repo_id='m-aliabbas1/xtts-urdu-voice-v0')

In [51]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="m-aliabbas1/xtts-urdu-voice-v0",
    folder_path="./xtts-urdu-voice",
    repo_type="model",
    commit_message="Initial XTTS Urdu model upload"
)

sample_wav.wav:   0%|          | 0.00/273k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.pth:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/m-aliabbas1/xtts-urdu-voice-v0/commit/3115de612411021ad413eb0d8d677878d56f1be8', commit_message='Initial XTTS Urdu model upload', commit_description='', oid='3115de612411021ad413eb0d8d677878d56f1be8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/m-aliabbas1/xtts-urdu-voice-v0', endpoint='https://huggingface.co', repo_type='model', repo_id='m-aliabbas1/xtts-urdu-voice-v0'), pr_revision=None, pr_num=None)